# Density Matrix Renormalization Group

## DMRG function from ITensors

In [1]:
using ITensors

In [3]:
N = 10
sites = siteinds("S=1", N)     # S=1 spin chain

# nearest neighbor Heisenberg model
os = OpSum()
for j=1:N-1
    os += "Sz",j,"Sz",j+1
    os += 1/2,"S+",j,"S-",j+1
    os += 1/2,"S-",j,"S+",j+1
end
H = MPO(os, sites)

# initialized MPS state with bond dimension of 10
ψ0 = randomMPS(sites, 10)

# 5 sweep to get ground state and energy
num_sweeps = 5
max_dim = [10, 20, 100, 100, 100]
cutoff = 1e-10
energy, ψ = dmrg(H, ψ0; nsweeps=num_sweeps, maxdim=max_dim, cutoff=cutoff)
energy

After sweep 1 energy=-138.81329497491564  maxlinkdim=10 maxerr=1.26E-02 time=1.159


After sweep 2 energy=-138.93726539750824  maxlinkdim=20 maxerr=5.03E-06 time=3.506


After sweep 3 energy=-138.94008428301743  maxlinkdim=90 maxerr=9.99E-11 time=53.384


After sweep 4 energy=-138.94008605519397  maxlinkdim=99 maxerr=9.99E-11 time=166.310


After sweep 5 energy=-138.9400860543637  maxlinkdim=95 maxerr=9.96E-11 time=179.205


-138.9400860543637

## Write a DMRG code based on ITensors
The package ITensors provides method to construct MPS states automatically, and thus we can use the package to design our own DMRG function. 

In [1]:
using ITensors
using KrylovKit
using Printf
using ProgressMeter

function SimpleDMRG(H, ψ₀, num_sweeps, max_dim, cutoff)
    ψ = copy(ψ₀)
    N = length(ψ)

    # warm up
    PH = ProjMPO(H)
    PH = position!(PH, ψ, 1)
    @printf "Start sweeping:\n"
    flush(stdout)

    energy = 0
    # sweep
    for i in 1:num_sweeps
        p = Progress((N-1)*2; dt=1,
            desc="Sweep " * string(i) * ": ",
            barglyphs=BarGlyphs('|','█', ['▁' ,'▂' ,'▃' ,'▄' ,'▅' ,'▆', '▇'],' ','|',),
            barlen=10)

        # left to right
        for j in 1:N-1
            PH = position!(PH, ψ, j)
            local_ψ = ψ[j] * ψ[j+1]

            # choose lowest energy state
            D, U = eigsolve(
                    PH,
                    local_ψ,
                    1,
                    :SR;
                    ishermitian=true,
                    tol=1e-14,
                    krylovdim=3,
                    maxiter=1,
                  )
            local_ψ = U[1]

            # trancate dimension
            left_ind = inds(ψ[j])
            L, R, spec = factorize(local_ψ, left_ind,
                which_decomp="svd", ortho="left",
                cutoff=cutoff, maxdim=max_dim[i], tags=tags(linkind(ψ, j)))
            ψ[j] = L
            ψ[j+1] = R

            # normalize
            ψ[j+1] ./= norm(ψ[j+1])

            energy = D[1]
            next!(p; showvalues=[(:energy, round(real(energy), digits=4))])
        end

        # right to left
        for j in N:-1:2
            PH = position!(PH, ψ, j-1)

            local_ψ = ψ[j] * ψ[j-1]

            # choose lowest energy state
            D, U = eigsolve(
                    PH,
                    local_ψ,
                    1,
                    :SR;
                    ishermitian=true,
                    tol=1e-14,
                    krylovdim=3,
                    maxiter=1,
                  )
            local_ψ = U[1]

            # trancate dimension
            left_ind = inds(ψ[j-1])
            L, R, spec = factorize(local_ψ, left_ind,
                which_decomp="svd", ortho="right",
                cutoff=cutoff, maxdim=max_dim[i], tags=tags(linkind(ψ, j-1)))
            ψ[j] = R
            ψ[j-1] = L

            # normalize
            ψ[j-1] ./= norm(ψ[j-1])

            energy = D[1]
            next!(p; showvalues=[(:energy, round(real(energy), digits=4))])
        end


        finish!(p)
    end
end

SimpleDMRG (generic function with 1 method)

In [2]:
N = 100
sites = siteinds("S=1", N)     # S=1 spin chain

# nearest neighbor Heisenberg model
os = OpSum()
for j=1:N-1
    os += "Sz",j,"Sz",j+1
    os += 1/2,"S+",j,"S-",j+1
    os += 1/2,"S-",j,"S+",j+1
end
H = MPO(os, sites)

# initialized MPS state with bond dimension of 10
ψ0 = randomMPS(sites, 10)

# 5 sweep to get ground state and energy
num_sweeps = 2
max_dim = [10, 20, 100, 100, 100]
cutoff = 1e-10;

In [3]:
@time SimpleDMRG(H, ψ0, num_sweeps, max_dim, cutoff)

Start sweeping:


Sweep 1:   1%|▁         |  ETA: 0:35:59



Sweep 1:  52%|█████▁    |  ETA: 0:00:23



Sweep 1: 100%|██████████| Time: 0:00:25
  energy:  -138.7956


Sweep 2:  43%|████▂     |  ETA: 0:00:02



Sweep 2:  79%|███████▇  |  ETA: 0:00:01



Sweep 2:  95%|█████████▄|  ETA: 0:00:00



Sweep 2: 100%|██████████| Time: 0:00:04
  energy:  -138.9373


 46.336217 seconds (90.48 M allocations: 6.123 GiB, 5.45% gc time, 86.81% compilation time: <1% of which was recompilation)
